In [1]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import seaborn as sns
from sklearn.feature_selection import SelectKBest,f_classif

In [2]:
import warnings
warnings.simplefilter("ignore")

# LOAD DATA

In [3]:
neutral = ["N"]*15
happy = ["H"] *15
anger = ["A"] *15
L = neutral+happy+anger
def load_noise_data(FILIP_PATH,noise_level,no_of_sentences,which):
    data = [i for i in range(no_of_sentences)]
    for i in range(1, no_of_sentences+1):
        
        file_name = "sentence"+str(i)+L[i-1]+"_"+which+"_"+noise_level+".wav"
        y, sr = librosa.load(FILIP_PATH+file_name, sr=None)
        data[i-1] = y
    return data

In [4]:
DATA_PATH = "BazaNoise/"
FILIP_PATH = DATA_PATH + "sentencesFilipNoise/"
SEB_PATH = DATA_PATH + "sentencesSebastianNoise/"
MARCIN_PATH = DATA_PATH + "sentencesMarcinNoise/"
PROF_PATH = DATA_PATH + "sentencesProfessorNoise/"
no_of_sentences = 45
Fs = 48000


## Functions to get mfcc features

In [5]:
def get_mfcc_mean_features(data,no_of_sentences, mfcc_features_no):
    mfcc = [i for i in range(no_of_sentences)]
    mfcc_delta = [i for i in range(no_of_sentences)]
    mfcc_delta2 = [i for i in range(no_of_sentences)]

    for i, word in enumerate(data):
        mfcc[i] = librosa.feature.mfcc(y=word, sr=Fs, n_mfcc=mfcc_features_no)
        mfcc_delta[i] = librosa.feature.delta(word)
        mfcc_delta2[i] =  librosa.feature.delta(word, order=2)

    mfcc_mean = np.zeros((no_of_sentences,mfcc_features_no))
    for i in range(no_of_sentences):
        mfcc_mean[i,0:mfcc_features_no] = np.mean(mfcc[i],axis=1)
    
    return mfcc_mean

def get_mfcc_features(data,mfcc_features_no):
    mfcc = [i for i in range(no_of_sentences)]
    mfcc_delta = [i for i in range(no_of_sentences)]
    mfcc_delta2 = [i for i in range(no_of_sentences)]

    for i, word in enumerate(data):
        mfcc[i] = librosa.feature.mfcc(y=word, sr=Fs, n_mfcc=mfcc_features_no)
        mfcc_delta[i] = librosa.feature.delta(word)
        mfcc_delta2[i] =  librosa.feature.delta(word, order=2)
        
    return mfcc

# Functions to get spectal features

In [6]:
def get_spectral_mean_features(data, no_of_sentences,spectral_features_no = 4):
    spectral_features = [i for i in range(no_of_sentences)]
    sr = 48000
    for i, word in enumerate(data):
        spectral_features_vector = np.zeros(4)
        spectral_features_vector[0] = np.mean(librosa.feature.spectral_centroid(y=word, sr=sr),axis=1)[0]
        spectral_features_vector[1] = np.mean(librosa.feature.spectral_bandwidth(y=word, sr=sr),axis=1)[0]
        spectral_features_vector[2] = np.mean(librosa.feature.spectral_contrast(y=word, sr=sr),axis=1)[0]
        spectral_features_vector[3] = np.mean(librosa.feature.spectral_flatness(y=word),axis=1)[0]
        spectral_features[i] = spectral_features_vector
    return spectral_features

## Functions to get zero crossing

In [7]:
def get_zero_crossing_features(data, no_of_sentences) :
    zcr_mean = np.zeros((no_of_sentences, 1))
    for i in range(no_of_sentences):
        zcr_mean[i, 0] = np.mean(data[i])
    return zcr_mean

## Functions to get chroma features

In [8]:
def get_mean_chroma_features(data, chroma_types):

    chroma_stft = []
    chroma_cqt = []
    chroma_cens = []
    chroma_vqt = []

    sr = 48000

    for audio in data:
        word_stft = chroma_types[list(chroma_types.keys())[0]]["func"](y=audio, sr=sr, **chroma_types[list(chroma_types.keys())[0]]["params"])
        word_cqt = chroma_types[list(chroma_types.keys())[1]]["func"](y=audio, sr=sr, **chroma_types[list(chroma_types.keys())[1]]["params"])
        word_cens = chroma_types[list(chroma_types.keys())[2]]["func"](y=audio, sr=sr, **chroma_types[list(chroma_types.keys())[2]]["params"])
        word_vqt = chroma_types[list(chroma_types.keys())[3]]["func"](y=audio, sr=sr, **chroma_types[list(chroma_types.keys())[3]]["params"])


        chroma_stft.append(word_stft.mean(axis=1).flatten())
        chroma_cqt.append(word_cqt.mean(axis=1).flatten())
        chroma_cens.append(word_cens.mean(axis=1).flatten())
        chroma_vqt.append(word_vqt.mean(axis=1).flatten())

    return chroma_stft, chroma_cqt, chroma_cens, chroma_vqt

chroma_types ={
    "stft": {
        "func": librosa.feature.chroma_stft,
        "params": {},
    },
    "cqt": {
        "func": librosa.feature.chroma_cqt,
        "params": {},
    },
    "cens": {
        "func": librosa.feature.chroma_cens,
        "params": {},
    },
    "vqt": {
        "func": librosa.feature.chroma_vqt,
        "params": {"intervals": "equal"},
    }
}

# Functions to get classifier results and show confusion matrix

In [9]:
def get_clf_results(clf, X_train, X_test, y_train, y_test):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    return accuracy_score(y_test, y_pred), classification_report(y_test, y_pred), y_pred

def show_conf_matirx(y_test, y_pred,clf,feature ):
    cm = confusion_matrix(y_test, y_pred)
    ticks = ["Filip","Sebastian","Marcin","Łazoryszczak"]

    sns.heatmap(cm, annot=True,fmt='d', cmap='YlGnBu',xticklabels=ticks, yticklabels=ticks )
    plt.ylabel('Prediction',fontsize=12)
    plt.xlabel('Actual',fontsize=12)
    plt.title('Confusion Matrix - ' + feature +" - "+type(clf).__name__,fontsize=16)
    plt.show()

## Classifiers

# Getting full features

In [10]:
def get_feature_vector_mfcc(words_filip,words_seb,words_marcin, words_prof, no_of_sentences, mfcc_features_no):
    mfcc_filip_mean = get_mfcc_mean_features(words_filip,no_of_sentences,mfcc_features_no)
    mfcc_seb_mean = get_mfcc_mean_features(words_seb,no_of_sentences,mfcc_features_no)
    mfcc_marcin_mean = get_mfcc_mean_features(words_marcin,no_of_sentences,mfcc_features_no)
    mfcc_prof_mean = get_mfcc_mean_features(words_prof,no_of_sentences,mfcc_features_no)

    feature_vector_mfcc = np.concatenate((mfcc_filip_mean, mfcc_seb_mean,mfcc_marcin_mean,mfcc_prof_mean))

    return feature_vector_mfcc

def get_feature_vector_spectral(words_filip,words_seb,words_marcin, words_prof, no_of_sentences,spectral_features_no):
    spectral_features_filip = get_spectral_mean_features(words_filip, no_of_sentences,spectral_features_no)
    spectral_features_seb = get_spectral_mean_features(words_seb, no_of_sentences,spectral_features_no)
    spectral_features_marcin = get_spectral_mean_features(words_marcin, no_of_sentences,spectral_features_no)
    spectral_features_prof = get_spectral_mean_features(words_prof, no_of_sentences,spectral_features_no)

    feature_vector_spectral = np.concatenate((spectral_features_filip, spectral_features_seb,spectral_features_marcin,spectral_features_prof))

    return feature_vector_spectral

def get_feature_vector_zeros(words_filip,words_seb,words_marcin, words_prof, no_of_sentences):
    zcr_filip  = get_zero_crossing_features(words_filip, no_of_sentences)
    zcr_seb    = get_zero_crossing_features(words_seb, no_of_sentences)
    zcr_marcin = get_zero_crossing_features(words_marcin, no_of_sentences)
    zcr_prof   = get_zero_crossing_features(words_prof, no_of_sentences)

    feature_vector_zeros = np.concatenate((zcr_filip, zcr_seb,zcr_marcin,zcr_prof))

    return feature_vector_zeros

def get_feature_vector_chroma(words_filip,words_seb,words_marcin, words_prof,chroma_types):
    chroma_filip_stft_mean, chroma_filip_cqt_mean, chroma_filip_cens_mean, chroma_filip_vqt_mean = get_mean_chroma_features(words_filip, chroma_types)
    chroma_seb_stft_mean, chroma_seb_cqt_mean, chroma_seb_cens_mean, chroma_seb_vqt_mean = get_mean_chroma_features(words_seb, chroma_types)
    chroma_marcin_stft_mean, chroma_marcin_cqt_mean, chroma_marcin_cens_mean, chroma_marcin_vqt_mean = get_mean_chroma_features(words_marcin, chroma_types)
    chroma_prof_stft_mean, chroma_prof_cqt_mean, chroma_prof_cens_mean, chroma_prof_vqt_mean = get_mean_chroma_features(words_prof, chroma_types)

    chroma_filip = np.concatenate((np.array(chroma_filip_stft_mean),np.array(chroma_filip_cqt_mean),np.array(chroma_filip_cens_mean),np.array(chroma_filip_vqt_mean)),axis=1)
    chroma_seb = np.concatenate((np.array(chroma_seb_stft_mean),np.array(chroma_seb_cqt_mean),np.array(chroma_seb_cens_mean),np.array(chroma_seb_vqt_mean)),axis=1)
    chroma_marcin = np.concatenate((np.array(chroma_marcin_stft_mean),np.array(chroma_marcin_cqt_mean),np.array(chroma_marcin_cens_mean),np.array(chroma_marcin_vqt_mean)),axis=1)
    chroma_prof = np.concatenate((np.array(chroma_prof_stft_mean),np.array(chroma_prof_cqt_mean),np.array(chroma_prof_cens_mean),np.array(chroma_prof_vqt_mean)),axis=1)

    feature_vector_chroma = np.concatenate((chroma_filip, chroma_seb,chroma_marcin,chroma_prof))

    return feature_vector_chroma


In [11]:
svm = SVC(kernel='rbf', C=1.0, random_state=42)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100,20), activation='relu', solver='adam', 
                         max_iter=3000, random_state=42)


def get_results_per_feature(feature_vector,feature_name):
    labels = L * 4
    X_train, X_test, y_train, y_test = train_test_split(feature_vector, labels, test_size=0.5, random_state=42)

    acc_score_svm, class_report_svm, y_pred_svm = get_clf_results(svm, X_train, X_test, y_train, y_test)
    acc_score_rbf, class_report_rbf, y_pred_rbf= get_clf_results(rf_clf, X_train, X_test, y_train, y_test)
    acc_score_mlp, class_report_mlp, y_pred_mlp = get_clf_results(mlp_clf, X_train, X_test, y_train, y_test)

    print(feature_name)
    print("SVM", acc_score_svm)#, class_report_svm)
    print("Random Forest",acc_score_rbf)#, class_report_rbf)
    print("MLP", acc_score_mlp)#, class_report_mlp)
    print()

    # show_conf_matirx(y_test, y_pred_svm,svm,feature_name)
    # show_conf_matirx(y_test, y_pred_rbf,rf_clf,feature_name)
    # show_conf_matirx(y_test, y_pred_mlp,mlp_clf,feature_name)
    

# Noise - 10%

In [30]:
noise_level = "N10"

words_filip = load_noise_data(FILIP_PATH,noise_level,no_of_sentences, "F")
words_seb = load_noise_data(SEB_PATH,noise_level,no_of_sentences, "S")
words_marcin = load_noise_data(MARCIN_PATH,noise_level,no_of_sentences, "M")
words_prof = load_noise_data(PROF_PATH,noise_level,no_of_sentences, "L")


mfcc_features_no = 13
spectral_features_no = 4

feature_vector_mfcc = get_feature_vector_mfcc(words_filip,words_seb,words_marcin,words_prof,no_of_sentences, mfcc_features_no)
feature_vector_spectral = get_feature_vector_spectral(words_filip,words_seb,words_marcin,words_prof,no_of_sentences, spectral_features_no)
feature_vector_zeros = get_feature_vector_zeros(words_filip, words_seb,words_marcin,words_prof,no_of_sentences)
feature_vector_chroma = get_feature_vector_chroma(words_filip, words_seb,words_marcin,words_prof,chroma_types)


In [31]:
print(feature_vector_mfcc.shape)
get_results_per_feature(feature_vector_mfcc,"mfcc")
get_results_per_feature(feature_vector_spectral,"Spectral")
get_results_per_feature(feature_vector_zeros,"Zero Crossing")
get_results_per_feature(feature_vector_chroma,"Chroma")

(180, 13)
mfcc
SVM 0.3333333333333333
Random Forest 0.7777777777777778
MLP 0.36666666666666664

Spectral
SVM 0.3
Random Forest 0.5222222222222223
MLP 0.34444444444444444

Zero Crossing
SVM 0.34444444444444444
Random Forest 0.3333333333333333
MLP 0.3

Chroma
SVM 0.5
Random Forest 0.6666666666666666
MLP 0.6444444444444445



# Noise - 20%

In [14]:
noise_level = "N20"

words_filip = load_noise_data(FILIP_PATH,noise_level,no_of_sentences, "F")
words_seb = load_noise_data(SEB_PATH,noise_level,no_of_sentences, "S")
words_marcin = load_noise_data(MARCIN_PATH,noise_level,no_of_sentences, "M")
words_prof = load_noise_data(PROF_PATH,noise_level,no_of_sentences, "L")


mfcc_features_no = 13
spectral_features_no = 4

feature_vector_mfcc = get_feature_vector_mfcc(words_filip,words_seb,words_marcin,words_prof,no_of_sentences, mfcc_features_no)
feature_vector_spectral = get_feature_vector_spectral(words_filip,words_seb,words_marcin,words_prof,no_of_sentences, spectral_features_no)
feature_vector_zeros = get_feature_vector_zeros(words_filip, words_seb,words_marcin,words_prof,no_of_sentences)
feature_vector_chroma = get_feature_vector_chroma(words_filip, words_seb,words_marcin,words_prof,chroma_types)

In [15]:
get_results_per_feature(feature_vector_mfcc,"mfcc")
get_results_per_feature(feature_vector_spectral,"Spectral")
get_results_per_feature(feature_vector_zeros,"Zero Crossing")
get_results_per_feature(feature_vector_chroma,"Chroma")

mfcc
SVM 0.3
Random Forest 0.7444444444444445
MLP 0.7333333333333333

Spectral
SVM 0.32222222222222224
Random Forest 0.5333333333333333
MLP 0.5

Zero Crossing
SVM 0.3111111111111111
Random Forest 0.28888888888888886
MLP 0.3

Chroma
SVM 0.4888888888888889
Random Forest 0.6222222222222222
MLP 0.6666666666666666



# Noise - 30 %

In [16]:
noise_level = "N30"

words_filip = load_noise_data(FILIP_PATH,noise_level,no_of_sentences, "F")
words_seb = load_noise_data(SEB_PATH,noise_level,no_of_sentences, "S")
words_marcin = load_noise_data(MARCIN_PATH,noise_level,no_of_sentences, "M")
words_prof = load_noise_data(PROF_PATH,noise_level,no_of_sentences, "L")


mfcc_features_no = 13
spectral_features_no = 4

feature_vector_mfcc = get_feature_vector_mfcc(words_filip,words_seb,words_marcin,words_prof,no_of_sentences, mfcc_features_no)
feature_vector_spectral = get_feature_vector_spectral(words_filip,words_seb,words_marcin,words_prof,no_of_sentences, spectral_features_no)
feature_vector_zeros = get_feature_vector_zeros(words_filip, words_seb,words_marcin,words_prof,no_of_sentences)
feature_vector_chroma = get_feature_vector_chroma(words_filip, words_seb,words_marcin,words_prof,chroma_types)

In [17]:
get_results_per_feature(feature_vector_mfcc,"mfcc")
get_results_per_feature(feature_vector_spectral,"Spectral")
get_results_per_feature(feature_vector_zeros,"Zero Crossing")
get_results_per_feature(feature_vector_chroma,"Chroma")

mfcc
SVM 0.3
Random Forest 0.7
MLP 0.6444444444444445

Spectral
SVM 0.3111111111111111
Random Forest 0.4888888888888889
MLP 0.5555555555555556

Zero Crossing
SVM 0.36666666666666664
Random Forest 0.32222222222222224
MLP 0.3

Chroma
SVM 0.5333333333333333
Random Forest 0.6888888888888889
MLP 0.6888888888888889



# Noise - 40%

In [18]:
noise_level = "N40"

words_filip = load_noise_data(FILIP_PATH,noise_level,no_of_sentences, "F")
words_seb = load_noise_data(SEB_PATH,noise_level,no_of_sentences, "S")
words_marcin = load_noise_data(MARCIN_PATH,noise_level,no_of_sentences, "M")
words_prof = load_noise_data(PROF_PATH,noise_level,no_of_sentences, "L")


mfcc_features_no = 13
spectral_features_no = 4

feature_vector_mfcc = get_feature_vector_mfcc(words_filip,words_seb,words_marcin,words_prof,no_of_sentences, mfcc_features_no)
feature_vector_spectral = get_feature_vector_spectral(words_filip,words_seb,words_marcin,words_prof,no_of_sentences, spectral_features_no)
feature_vector_zeros = get_feature_vector_zeros(words_filip, words_seb,words_marcin,words_prof,no_of_sentences)
feature_vector_chroma = get_feature_vector_chroma(words_filip, words_seb,words_marcin,words_prof,chroma_types)

In [19]:
get_results_per_feature(feature_vector_mfcc,"mfcc")
get_results_per_feature(feature_vector_spectral,"Spectral")
get_results_per_feature(feature_vector_zeros,"Zero Crossing")
get_results_per_feature(feature_vector_chroma,"Chroma")

mfcc
SVM 0.3
Random Forest 0.6333333333333333
MLP 0.7333333333333333

Spectral
SVM 0.3
Random Forest 0.5222222222222223
MLP 0.4666666666666667

Zero Crossing
SVM 0.36666666666666664
Random Forest 0.37777777777777777
MLP 0.3

Chroma
SVM 0.5666666666666667
Random Forest 0.7333333333333333
MLP 0.6555555555555556



# Noise - 50%

In [20]:
noise_level = "N50"

words_filip = load_noise_data(FILIP_PATH,noise_level,no_of_sentences, "F")
words_seb = load_noise_data(SEB_PATH,noise_level,no_of_sentences, "S")
words_marcin = load_noise_data(MARCIN_PATH,noise_level,no_of_sentences, "M")
words_prof = load_noise_data(PROF_PATH,noise_level,no_of_sentences, "L")


mfcc_features_no = 13
spectral_features_no = 4

feature_vector_mfcc = get_feature_vector_mfcc(words_filip,words_seb,words_marcin,words_prof,no_of_sentences, mfcc_features_no)
feature_vector_spectral = get_feature_vector_spectral(words_filip,words_seb,words_marcin,words_prof,no_of_sentences, spectral_features_no)
feature_vector_zeros = get_feature_vector_zeros(words_filip, words_seb,words_marcin,words_prof,no_of_sentences)
feature_vector_chroma = get_feature_vector_chroma(words_filip, words_seb,words_marcin,words_prof,chroma_types)

In [21]:
get_results_per_feature(feature_vector_mfcc,"mfcc")
get_results_per_feature(feature_vector_spectral,"Spectral")
get_results_per_feature(feature_vector_zeros,"Zero Crossing")
get_results_per_feature(feature_vector_chroma,"Chroma")

mfcc
SVM 0.3
Random Forest 0.6444444444444445
MLP 0.7

Spectral
SVM 0.3
Random Forest 0.4666666666666667
MLP 0.3

Zero Crossing
SVM 0.4
Random Forest 0.34444444444444444
MLP 0.3

Chroma
SVM 0.5777777777777777
Random Forest 0.7444444444444445
MLP 0.6777777777777778



# Noise - 60%

In [22]:
noise_level = "N60"

words_filip = load_noise_data(FILIP_PATH,noise_level,no_of_sentences, "F")
words_seb = load_noise_data(SEB_PATH,noise_level,no_of_sentences, "S")
words_marcin = load_noise_data(MARCIN_PATH,noise_level,no_of_sentences, "M")
words_prof = load_noise_data(PROF_PATH,noise_level,no_of_sentences, "L")


mfcc_features_no = 13
spectral_features_no = 4

feature_vector_mfcc = get_feature_vector_mfcc(words_filip,words_seb,words_marcin,words_prof,no_of_sentences, mfcc_features_no)
feature_vector_spectral = get_feature_vector_spectral(words_filip,words_seb,words_marcin,words_prof,no_of_sentences, spectral_features_no)
feature_vector_zeros = get_feature_vector_zeros(words_filip, words_seb,words_marcin,words_prof,no_of_sentences)
feature_vector_chroma = get_feature_vector_chroma(words_filip, words_seb,words_marcin,words_prof,chroma_types)

In [23]:
get_results_per_feature(feature_vector_mfcc,"mfcc")
get_results_per_feature(feature_vector_spectral,"Spectral")
get_results_per_feature(feature_vector_zeros,"Zero Crossing")
get_results_per_feature(feature_vector_chroma,"Chroma")

mfcc
SVM 0.3
Random Forest 0.6777777777777778
MLP 0.6111111111111112

Spectral
SVM 0.3
Random Forest 0.5
MLP 0.5

Zero Crossing
SVM 0.35555555555555557
Random Forest 0.35555555555555557
MLP 0.3

Chroma
SVM 0.4111111111111111
Random Forest 0.7
MLP 0.6222222222222222



# Noise - 70%

In [24]:
noise_level = "N70"

words_filip = load_noise_data(FILIP_PATH,noise_level,no_of_sentences, "F")
words_seb = load_noise_data(SEB_PATH,noise_level,no_of_sentences, "S")
words_marcin = load_noise_data(MARCIN_PATH,noise_level,no_of_sentences, "M")
words_prof = load_noise_data(PROF_PATH,noise_level,no_of_sentences, "L")


mfcc_features_no = 13
spectral_features_no = 4

feature_vector_mfcc = get_feature_vector_mfcc(words_filip,words_seb,words_marcin,words_prof,no_of_sentences, mfcc_features_no)
feature_vector_spectral = get_feature_vector_spectral(words_filip,words_seb,words_marcin,words_prof,no_of_sentences, spectral_features_no)
feature_vector_zeros = get_feature_vector_zeros(words_filip, words_seb,words_marcin,words_prof,no_of_sentences)
feature_vector_chroma = get_feature_vector_chroma(words_filip, words_seb,words_marcin,words_prof,chroma_types)

In [25]:
get_results_per_feature(feature_vector_mfcc,"mfcc")
get_results_per_feature(feature_vector_spectral,"Spectral")
get_results_per_feature(feature_vector_zeros,"Zero Crossing")
get_results_per_feature(feature_vector_chroma,"Chroma")

mfcc
SVM 0.3
Random Forest 0.7333333333333333
MLP 0.6666666666666666

Spectral
SVM 0.3
Random Forest 0.5111111111111111
MLP 0.4444444444444444

Zero Crossing
SVM 0.32222222222222224
Random Forest 0.28888888888888886
MLP 0.3

Chroma
SVM 0.32222222222222224
Random Forest 0.6666666666666666
MLP 0.5333333333333333



# Nosie - 80 %

In [26]:
noise_level = "N80"

words_filip = load_noise_data(FILIP_PATH,noise_level,no_of_sentences, "F")
words_seb = load_noise_data(SEB_PATH,noise_level,no_of_sentences, "S")
words_marcin = load_noise_data(MARCIN_PATH,noise_level,no_of_sentences, "M")
words_prof = load_noise_data(PROF_PATH,noise_level,no_of_sentences, "L")


mfcc_features_no = 13
spectral_features_no = 4

feature_vector_mfcc = get_feature_vector_mfcc(words_filip,words_seb,words_marcin,words_prof,no_of_sentences, mfcc_features_no)
feature_vector_spectral = get_feature_vector_spectral(words_filip,words_seb,words_marcin,words_prof,no_of_sentences, spectral_features_no)
feature_vector_zeros = get_feature_vector_zeros(words_filip, words_seb,words_marcin,words_prof,no_of_sentences)
feature_vector_chroma = get_feature_vector_chroma(words_filip, words_seb,words_marcin,words_prof,chroma_types)

In [27]:
get_results_per_feature(feature_vector_mfcc,"mfcc")
get_results_per_feature(feature_vector_spectral,"Spectral")
get_results_per_feature(feature_vector_zeros,"Zero Crossing")
get_results_per_feature(feature_vector_chroma,"Chroma")

mfcc
SVM 0.3
Random Forest 0.7111111111111111
MLP 0.6222222222222222

Spectral
SVM 0.3
Random Forest 0.5333333333333333
MLP 0.36666666666666664

Zero Crossing
SVM 0.3333333333333333
Random Forest 0.36666666666666664
MLP 0.3

Chroma
SVM 0.32222222222222224
Random Forest 0.5222222222222223
MLP 0.5



# Noise - 90 %

In [28]:
noise_level = "N90"

words_filip = load_noise_data(FILIP_PATH,noise_level,no_of_sentences, "F")
words_seb = load_noise_data(SEB_PATH,noise_level,no_of_sentences, "S")
words_marcin = load_noise_data(MARCIN_PATH,noise_level,no_of_sentences, "M")
words_prof = load_noise_data(PROF_PATH,noise_level,no_of_sentences, "L")


mfcc_features_no = 13
spectral_features_no = 4

feature_vector_mfcc = get_feature_vector_mfcc(words_filip,words_seb,words_marcin,words_prof,no_of_sentences, mfcc_features_no)
feature_vector_spectral = get_feature_vector_spectral(words_filip,words_seb,words_marcin,words_prof,no_of_sentences, spectral_features_no)
feature_vector_zeros = get_feature_vector_zeros(words_filip, words_seb,words_marcin,words_prof,no_of_sentences)
feature_vector_chroma = get_feature_vector_chroma(words_filip, words_seb,words_marcin,words_prof,chroma_types)

In [29]:
get_results_per_feature(feature_vector_mfcc,"mfcc")
get_results_per_feature(feature_vector_spectral,"Spectral")
get_results_per_feature(feature_vector_zeros,"Zero Crossing")
get_results_per_feature(feature_vector_chroma,"Chroma")

mfcc
SVM 0.3
Random Forest 0.6222222222222222
MLP 0.5444444444444444

Spectral
SVM 0.3
Random Forest 0.3888888888888889
MLP 0.34444444444444444

Zero Crossing
SVM 0.35555555555555557
Random Forest 0.4
MLP 0.3

Chroma
SVM 0.3111111111111111
Random Forest 0.4777777777777778
MLP 0.36666666666666664

